In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from glasses.nn.models.classification import *
from glasses.utils.Storage import ForwardModuleStorage
from glasses.nn.models.segmentation.unet import *
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet, ResNetBottleneckBlock
from glasses.nn.models.classification.efficientnet import *
from glasses.nn.models.classification import *
from torch import nn

In [3]:
from torchvision.models import resnet50


resnet50()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
unet = UNet(encoder=lambda *args, **kwargs: ResNet.resnet26(*args, **kwargs).encoder)

unet.summary((1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           3,136
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
         Conv2dPad-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
        Conv2dPad-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
        Conv2dPad-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(20893282), tensor(20893282), tensor(79.7015), tensor(345.7562))

# Benchmark

In [ ]:
import timm
from glasses.nn.models import *
timm.create_model('seresnet50')

In [ ]:
SEResNet.cse_resnet50()

In [ ]:
enc = EfficientNet.efficientnet_b0().encoder
x = torch.randn((1,3,224,224))

In [ ]:
from glasses.nn.models.classification.efficientnet import *
import timm

from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [ ]:
def from_encoder(model_def, *args, **kwargs):
    model = model_def( *args, **kwargs)
    encoder = nn.Identity()
    if isinstance(model, ResNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  *model.encoder.layers,
                              ],
                              features_widths = model.encoder.widths[1:]
                             )
        
    elif isinstance(model, EfficientNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  model.encoder.layers[1],
                                  model.encoder.layers[2],
                                  model.encoder.layers[3],
                              ],
                              features_widths = [model.encoder.widths[0],
                                                 model.encoder.widths[2],
                                                 model.encoder.widths[3],
                                                 model.encoder.widths[4],
                                                ]
                             )
        
    return encoder

In [ ]:
enc = ResNet.resnet18().encoder

In [ ]:
enc.features_widths()

In [ ]:
enc.stages()

In [ ]:
enc.features

enc(x)

enc.features


In [ ]:
m

In [ ]:
backbone = EfficientNet.efficientnet_b0().encoder


encoder = WithFeatures(backbone, 
                              stages = [
                                  backbone.stem[-2], 
                                  backbone.layers[1],
                                  backbone.layers[2],
                                  backbone.layers[3],
                              ],
                              features_widths = [32, 24, 40, 80]
                             )


m = UNet(encoder = lambda *args, **kwargs: encoder, decoder = partial(UNetDecoder, 
                                                                      widths=[1280, 256, 128, 64, 32, 16]))

In [ ]:
enc.features

In [ ]:
name = 'resnet-asdsa'

name.startswith('resnet')

In [ ]:
m.summary((3, 224, 224))

In [ ]:
dst = clone_model(src, dst)

In [ ]:
dst.summary()

In [ ]:
dst

In [ ]:
# import torch

# transform = dst.configs['efficientnet_lite1'].transform

# benchmark(dst.cuda(), transform)

In [ ]:
# benchmark(src.cuda(), transform)

In [ ]:
src = timm.create_model('efficientnet_b0', pretrained='True')
src

In [ ]:
dst.summary()